<a href="https://colab.research.google.com/github/pangeab-blip/EvGeo-Exercises/blob/main/paleomag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%%writefile paleomag_colab_full.py
# paleomag_colab_full.py
# Paleomag Toolkit — full Colab app with interactive UI (Gradio)
# Protocolli: DIR2POLE, POLE2DIR, UNCERTAINTY, RFTEST, POLESHIFT, PALAT, SOMMAMTX, ROTATE
# Extra: BATCH CSV, TEST SUITE, equal-area plots (matplotlib: nessun colore esplicito, 1 grafico per figura)

import math
import sys
import io

def _ensure_libs():
    try:
        import gradio as gr  # noqa
        import pandas as pd  # noqa
        import matplotlib  # noqa
    except Exception:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "gradio>=4.44", "pandas>=2.0", "matplotlib>=3.7"])
_ensure_libs()
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------
# Utility & geometry
# -----------------------
d2r = math.radians
r2d = math.degrees
EPS = 1e-12
SMALL = 1e-8

def wrap_lon_180(lon_deg: float) -> float:
    x = (lon_deg + 180.0) % 360.0
    if x < 0:
        x += 360.0
    return x - 180.0

def wrap_lon_360(lon_deg: float) -> float:
    x = lon_deg % 360.0
    if x < 0:
        x += 360.0
    return x

def sph2cart(lat_deg: float, lon_deg: float):
    lat = d2r(lat_deg); lon = d2r(lon_deg)
    cl = math.cos(lat)
    return (cl*math.cos(lon), cl*math.sin(lon), math.sin(lat))

def cart2sph(x: float, y: float, z: float):
    r = math.sqrt(x*x + y*y + z*z) + EPS
    lat = math.asin(z/r); lon = math.atan2(y, x)
    return (r2d(lat), r2d(lon))

def great_circle_distance_deg(lat1, lon1, lat2, lon2):
    x1, y1, z1 = sph2cart(lat1, lon1)
    x2, y2, z2 = sph2cart(lat2, lon2)
    dot = max(-1.0, min(1.0, x1*x2 + y1*y2 + z1*z2))
    return r2d(math.acos(dot))

# -----------------------
# Dipole relations
# -----------------------
def inc_from_p(p_deg: float) -> float:
    p = d2r(p_deg)
    tp = math.tan(p)
    if abs(tp) < EPS:
        return 90.0 if p_deg > 0 else -90.0
    I = math.atan(2.0 / tp)
    return r2d(I)

def p_from_inc(I_deg: float) -> float:
    I = d2r(I_deg)
    tI = math.tan(I)
    if abs(tI) < EPS:
        return 90.0
    tp = 2.0 / tI
    p = math.atan(tp)
    return r2d(p)

def pole_longitude_from_dir(site_lat, site_lon, I_deg, D_deg):
    λs = d2r(site_lat); φs = d2r(site_lon)
    p = d2r(p_from_inc(I_deg))
    D = d2r(D_deg)
    E = (-math.sin(φs), math.cos(φs), 0.0)
    N = (-math.sin(λs)*math.cos(φs), -math.sin(λs)*math.sin(φs), math.cos(λs))
    U = (math.cos(λs)*math.cos(φs), math.cos(λs)*math.sin(φs), math.sin(λs))
    v_h = (math.cos(D)*N[0] + math.sin(D)*E[0],
           math.cos(D)*N[1] + math.sin(D)*E[1],
           math.cos(D)*N[2] + math.sin(D)*E[2])
    X0 = U
    vh = math.sqrt(v_h[0]**2 + v_h[1]**2 + v_h[2]**2) + EPS
    v_h = (v_h[0]/vh, v_h[1]/vh, v_h[2]/vh)
    kx = v_h[1]*X0[2] - v_h[2]*X0[1]
    ky = v_h[2]*X0[0] - v_h[0]*X0[2]
    kz = v_h[0]*X0[1] - v_h[1]*X0[0]
    kn = math.sqrt(kx*kx + ky*ky + kz*kz) + EPS
    kx, ky, kz = kx/kn, ky/kn, kz/kn
    c = math.cos(p); s = math.sin(p)
    kxX0 = (ky*X0[2] - kz*X0[1], kz*X0[0] - kx*X0[2], kx*X0[1] - ky*X0[0])
    Xp = (X0[0]*c + kxX0[0]*s, X0[1]*c + kxX0[1]*s, X0[2]*c + kxX0[2]*s)
    lat_p, lon_p = cart2sph(*Xp)
    lon_p = wrap_lon_180(lon_p)
    return lat_p, lon_p

def dir2pole(site_lat, site_lon, I_deg, D_deg):
    lat_p, lon_p = pole_longitude_from_dir(site_lat, site_lon, I_deg, D_deg)
    p = great_circle_distance_deg(site_lat, site_lon, lat_p, lon_p)
    return lat_p, lon_p, p

def pole2dir(pole_lat, pole_lon, site_lat, site_lon):
    p = great_circle_distance_deg(site_lat, site_lon, pole_lat, pole_lon)
    I = inc_from_p(p)
    φ1 = d2r(site_lon); λ1 = d2r(site_lat)
    φ2 = d2r(pole_lon); λ2 = d2r(pole_lat)
    dφ = φ2 - φ1
    X = math.cos(λ2)*math.sin(dφ)
    Y = math.cos(λ1)*math.sin(λ2) - math.sin(λ1)*math.cos(λ2)*math.cos(dφ)
    az = math.atan2(X, Y)
    D = (r2d(az) + 360.0) % 360.0
    return I, D, p

# -----------------------
# Euler matrices & SOMMAMTX
# -----------------------
def rodrigues_matrix(E, phi_rad):
    Ex, Ey, Ez = E
    c = math.cos(phi_rad); s = math.sin(phi_rad); C = 1.0 - c
    return [
        [Ex*Ex*C + c,     Ex*Ey*C - Ez*s, Ex*Ez*C + Ey*s],
        [Ey*Ex*C + Ez*s,  Ey*Ey*C + c,    Ey*Ez*C - Ex*s],
        [Ez*Ex*C - Ey*s,  Ez*Ey*C + Ex*s, Ez*Ez*C + c   ],
    ]

def matmul(A, B):
    return [
        [A[0][0]*B[0][0] + A[0][1]*B[1][0] + A[0][2]*B[2][0],
         A[0][0]*B[0][1] + A[0][1]*B[1][1] + A[0][2]*B[2][1],
         A[0][0]*B[0][2] + A[0][1]*B[1][2] + A[0][2]*B[2][2]],
        [A[1][0]*B[0][0] + A[1][1]*B[1][0] + A[1][2]*B[2][0],
         A[1][0]*B[0][1] + A[1][1]*B[1][1] + A[1][2]*B[2][1],
         A[1][0]*B[0][2] + A[1][1]*B[1][2] + A[1][2]*B[2][2]],
        [A[2][0]*B[0][0] + A[2][1]*B[1][0] + A[2][2]*B[2][0],
         A[2][0]*B[0][1] + A[2][1]*B[1][1] + A[2][2]*B[2][1],
         A[2][0]*B[0][2] + A[2][1]*B[1][2] + A[2][2]*B[2][2]],
    ]

def euler_to_axis(lat_deg, lon_deg):
    Ex, Ey, Ez = sph2cart(lat_deg, lon_deg)
    n = math.sqrt(Ex*Ex + Ey*Ey + Ez*Ez) + EPS
    return (Ex/n, Ey/n, Ez/n)

def axis_angle_from_matrix(T):
    ux = T[2][1] - T[1][2]
    uy = T[0][2] - T[2][0]
    uz = T[1][0] - T[0][1]
    s = math.sqrt(ux*ux + uy*uy + uz*uz)
    c = (T[0][0] + T[1][1] + T[2][2]) - 1.0
    omega = math.atan2(s, c)
    if abs(omega) < SMALL:
        return (1.0, 0.0, 0.0), 0.0
    if abs(math.pi - omega) < SMALL:
        # 180°: asse da diagonale maggiore
        diag = [ (T[0][0], 0), (T[1][1], 1), (T[2][2], 2) ]
        diag.sort(reverse=True); idx = diag[0][1]
        if idx == 0:
            x = math.sqrt(max(0.0, (T[0][0]+1.0)/2.0)); y = T[0][1]/(2.0*x+EPS); z = T[0][2]/(2.0*x+EPS)
        elif idx == 1:
            y = math.sqrt(max(0.0, (T[1][1]+1.0)/2.0)); x = T[0][1]/(2.0*y+EPS); z = T[1][2]/(2.0*y+EPS)
        else:
            z = math.sqrt(max(0.0, (T[2][2]+1.0)/2.0)); x = T[0][2]/(2.0*z+EPS); y = T[1][2]/(2.0*z+EPS)
        n = math.sqrt(x*x + y*y + z*z) + EPS
        return (x/n, y/n, z/n), omega
    Ex = ux / (2.0*math.sin(omega) + EPS)
    Ey = uy / (2.0*math.sin(omega) + EPS)
    Ez = uz / (2.0*math.sin(omega) + EPS)
    n = math.sqrt(Ex*Ex + Ey*Ey + Ez*Ez) + EPS
    return (Ex/n, Ey/n, Ez/n), omega

def somma_mtx(lat1, lon1, ang1, lat2, lon2, ang2):
    E1 = euler_to_axis(lat1, lon1)
    E2 = euler_to_axis(lat2, lon2)
    phi1 = d2r(ang1); phi2 = d2r(ang2)
    A1 = rodrigues_matrix(E1, phi1)
    A2 = rodrigues_matrix(E2, phi2)
    T = matmul(A2, A1)  # young × old
    E_tot, omega = axis_angle_from_matrix(T)
    lat, lon = cart2sph(*E_tot)
    if omega < 0:
        omega = -omega; lat = -lat; lon = lon + 180.0
    lon = wrap_lon_180(lon); lon_alt = wrap_lon_360(lon)
    return lat, lon, lon_alt, r2d(omega)

# -----------------------
# ROTATE (single & batch)
# -----------------------
def rotate_vector_about_axis(v, axis, omega_deg):
    Ex, Ey, Ez = axis; x, y, z = v
    phi = d2r(omega_deg); c = math.cos(phi); s = math.sin(phi)
    dot = Ex*x + Ey*y + Ez*z
    cross = (Ey*z - Ez*y, Ez*x - Ex*z, Ex*y - Ey*x)
    vx = x*c + cross[0]*s + Ex*dot*(1-c)
    vy = y*c + cross[1]*s + Ey*dot*(1-c)
    vz = z*c + cross[2]*s + Ez*dot*(1-c)
    return (vx, vy, vz)

def rotate_pole(lat_p, lon_p, e_lat, e_lon, omega_deg):
    v = sph2cart(lat_p, lon_p)
    axis = euler_to_axis(e_lat, e_lon)
    v_rot = rotate_vector_about_axis(v, axis, omega_deg)
    lat_r, lon_r = cart2sph(*v_rot)
    lon_r = wrap_lon_180(lon_r)
    return lat_r, lon_r, wrap_lon_360(lon_r)

# -----------------------
# Uncertainty & helpers
# -----------------------
def fisher_alpha95(N: int, k: float) -> float:
    if N < 2 or k <= 0:
        return float('nan')
    return r2d(math.acos(max(-1.0, min(1.0, 1 - (N - 1) / N * ( (20**(1/(N-1))) - 1 ) / k))))

def dir_uncertainty(alpha95: float, I_deg: float):
    Δ = alpha95
    w = max(0.1, math.cos(d2r(I_deg)))
    dI = Δ * w; dD = Δ / w
    return dI, dD

def pole_oval_from_A95(A95: float):
    return A95, A95, 0.0

def rftest(I_obs, D_obs, I_exp, D_exp):
    R = (D_obs - D_exp + 180.0) % 360.0 - 180.0
    F = I_exp - I_obs
    return R, F

def poleshift(lat_p_obs, lon_p_obs, lat_p_ref, lon_p_ref, site_lat=None, site_lon=None):
    if site_lat is not None and site_lon is not None:
        p_obs = great_circle_distance_deg(site_lat, site_lon, lat_p_obs, lon_p_obs)
        p_ref = great_circle_distance_deg(site_lat, site_lon, lat_p_ref, lon_p_ref)
        dp = p_obs - p_ref
        return dp, p_obs, p_ref
    else:
        arc = great_circle_distance_deg(lat_p_obs, lon_p_obs, lat_p_ref, lon_p_ref)
        return arc, None, None

def palat_from_dir(I_deg): return r2d(math.atan(0.5 * math.tan(d2r(I_deg))))
def palat_from_pole(p_deg): return 90.0 - p_deg

# -----------------------
# UI callbacks
# -----------------------
def ui_DIR2POLE(lat_site, lon_site, inc_obs, dec_obs, a95):
    try:
        lat_p, lon_p, p = dir2pole(lat_site, lon_site, inc_obs, dec_obs)
        lon_p_alt = wrap_lon_360(lon_p)
        extra = "No uncertainty provided."
        if a95 and a95 > 0:
            dp, dm, az = pole_oval_from_A95(a95)
            extra = f"dp≈{dp:.2f}°, dm≈{dm:.2f}°, az≈{az:.1f}°"
        return (f"{lat_p:.4f}°", f"{lon_p:.4f}°  ({lon_p_alt:.4f}°E)", f"{p:.4f}°", extra)
    except Exception as e:
        return ("ERR", "ERR", "ERR", f"Exception: {e}")

def ui_POLE2DIR(lat_p, lon_p, lat_site, lon_site, A95):
    try:
        I, D, p = pole2dir(lat_p, lon_p, lat_site, lon_site)
        extra = f"p={p:.4f}°; no A95"
        if A95 and A95 > 0:
            dp, dm, az = pole_oval_from_A95(A95)
            extra = f"p={p:.4f}°, A95={A95:.2f}° → oval(dp~{dp:.2f}, dm~{dm:.2f})"
        return (f"{I:.4f}°", f"{D:.4f}°", f"{p:.4f}°", extra)
    except Exception as e:
        return ("ERR", "ERR", "ERR", f"Exception: {e}")

def ui_UNCERT_dir(alpha95, Imean):
    try:
        dI, dD = dir_uncertainty(alpha95, Imean)
        return (f"{dI:.3f}°", f"{dD:.3f}°")
    except Exception:
        return ("ERR", "ERR")

def ui_RFTEST(I_obs, D_obs, I_exp, D_exp, alpha95):
    try:
        R, F = rftest(I_obs, D_obs, I_exp, D_exp)
        dI, dD = (0.0, 0.0)
        if alpha95 and alpha95 > 0:
            dI, dD = dir_uncertainty(alpha95, I_exp)
        return (f"{R:.3f}°", f"{F:.3f}°", f"ΔI≈{dI:.2f}°, ΔD≈{dD:.2f}° (from α95={alpha95:.2f}°)")
    except Exception as e:
        return ("ERR", "ERR", f"Exception: {e}")

def ui_POLESHIFT(lat_obs, lon_obs, lat_ref, lon_ref, lat_site, lon_site):
    try:
        if lat_site is None or lon_site is None:
            arc, _, _ = poleshift(lat_obs, lon_obs, lat_ref, lon_ref, None, None)
            return (f"pole-to-pole arc: {arc:.3f}°", "N/A", "N/A")
        else:
            dp, p_obs, p_ref = poleshift(lat_obs, lon_obs, lat_ref, lon_ref, lat_site, lon_site)
            return (f"{dp:.3f}°", f"p_obs={p_obs:.3f}°", f"p_ref={p_ref:.3f}°")
    except Exception as e:
        return (f"Exception: {e}", "ERR", "ERR")

def ui_PALAT_from_dir(I):
    try:
        return f"{palat_from_dir(I):.3f}°"
    except Exception as e:
        return f"Exception: {e}"

def ui_PALAT_from_pole(p):
    try:
        return f"{palat_from_pole(p):.3f}°"
    except Exception as e:
        return f"Exception: {e}"

def ui_SOMMAMTX(lat1, lon1, ang1, lat2, lon2, ang2):
    try:
        lat, lon180, lon360, ang = somma_mtx(lat1, lon1, ang1, lat2, lon2, ang2)
        alt_lat = -lat; alt_lon180 = wrap_lon_180(lon180 + 180.0); alt_ang = -ang
        return (f"{lat:.4f}°", f"{lon180:.4f}°  ({lon360:.4f}°E)", f"{ang:.4f}°",
                f"{alt_lat:.4f}°", f"{alt_lon180:.4f}°", f"{alt_ang:.4f}°")
    except Exception as e:
        return ("ERR", "ERR", "ERR", "ERR", "ERR", f"Exception: {e}")

def ui_ROTATE_single(lat_p, lon_p, e_lat, e_lon, omega):
    try:
        rl, rlon180, rlon360 = rotate_pole(lat_p, lon_p, e_lat, e_lon, omega)
        return (f"{rl:.4f}°", f"{rlon180:.4f}°  ({rlon360:.4f}°E)")
    except Exception as e:
        return ("ERR", f"Exception: {e}")

def _detect_lat_lon_columns(df):
    cols = {c.lower(): c for c in df.columns}
    lat_col = None; lon_col = None
    for cand in ["lat", "latitude", "site_lat", "pole_lat"]:
        if cand in cols: lat_col = cols[cand]; break
    for cand in ["lon", "longitude", "site_lon", "pole_lon"]:
        if cand in cols: lon_col = cols[cand]; break
    if lat_col is None or lon_col is None:
        raise ValueError("CSV must contain columns named (lat, lon) or (latitude, longitude).")
    return lat_col, lon_col

def ui_ROTATE_batch(file_obj, e_lat, e_lon, omega):
    try:
        if file_obj is None:
            return "No file uploaded.", None
        df = pd.read_csv(file_obj.name)
        lat_col, lon_col = _detect_lat_lon_columns(df)
        out = df.copy()
        lat_rot = []; lon_rot_180 = []; lon_rot_360 = []
        for _, row in df.iterrows():
            try:
                rl, rlon180, rlon360 = rotate_pole(float(row[lat_col]), float(row[lon_col]), e_lat, e_lon, omega)
            except Exception:
                rl, rlon180, rlon360 = (float('nan'), float('nan'), float('nan'))
            lat_rot.append(rl); lon_rot_180.append(rlon180); lon_rot_360.append(rlon360)
        out["lat_rot"] = lat_rot; out["lon_rot"] = lon_rot_180; out["lon_rot_360E"] = lon_rot_360
        save_path = "/mnt/data/rotated_poles.csv"
        out.to_csv(save_path, index=False)
        return f"Processed {len(out)} rows. Saved to rotated_poles.csv", save_path
    except Exception as e:
        return f"Exception: {e}", None

# -----------------------
# BATCH dispatcher (CSV)
# -----------------------
def ui_BATCH(file_obj, op, site_lat, site_lon, A_lat, A_lon, A_ang, B_lat, B_lon, B_ang):
    try:
        if file_obj is None:
            return "No file uploaded.", None
        df = pd.read_csv(file_obj.name)
        out = df.copy()
        if op == "DIR2POLE":
            for col in ["site_lat","site_lon","inc","dec"]:
                if col not in df.columns:
                    return f"Missing column '{col}' for DIR2POLE.", None
            Lp=[]; Lop=[]; P=[]
            for _, row in df.iterrows():
                lp, lop, p = dir2pole(float(row["site_lat"]), float(row["site_lon"]),
                                      float(row["inc"]), float(row["dec"]))
                Lp.append(lp); Lop.append(wrap_lon_180(lop)); P.append(p)
            out["lat_pole"]=Lp; out["lon_pole"]=Lop; out["p"]=P
            save = "/mnt/data/batch_dir2pole.csv"
        elif op == "POLE2DIR":
            for col in ["pole_lat","pole_lon","site_lat","site_lon"]:
                if col not in df.columns:
                    return f"Missing column '{col}' for POLE2DIR.", None
            I=[]; D=[]; P=[]
            for _, row in df.iterrows():
                Ii, Dd, p = pole2dir(float(row["pole_lat"]), float(row["pole_lon"]),
                                     float(row["site_lat"]), float(row["site_lon"]))
                I.append(Ii); D.append(Dd); P.append(p)
            out["I"]=I; out["D"]=D; out["p"]=P
            save = "/mnt/data/batch_pole2dir.csv"
        elif op == "SOMMAMTX":
            for col in ["lat1","lon1","ang1","lat2","lon2","ang2"]:
                if col not in df.columns:
                    return f"Missing column '{col}' for SOMMAMTX.", None
            Lt=[]; Lo=[]; L360=[]; W=[]
            for _, row in df.iterrows():
                lat, lon, lon360, ang = somma_mtx(float(row["lat1"]), float(row["lon1"]), float(row["ang1"]),
                                                  float(row["lat2"]), float(row["lon2"]), float(row["ang2"]))
                Lt.append(lat); Lo.append(lon); L360.append(lon360); W.append(ang)
            out["lat_c"]=Lt; out["lon_c"]=Lo; out["lon_c_360E"]=L360; out["ang_c"]=W
            save = "/mnt/data/batch_somma.csv"
        elif op == "ROTATE":
            for col in ["lat","lon"]:
                if col not in df.columns:
                    return f"Missing column '{col}' for ROTATE.", None
            Lt=[]; Lo=[]; L360=[]
            for _, row in df.iterrows():
                rl, rlon, r360 = rotate_pole(float(row["lat"]), float(row["lon"]), A_lat, A_lon, A_ang)
                Lt.append(rl); Lo.append(rlon); L360.append(r360)
            out["lat_rot"]=Lt; out["lon_rot"]=Lo; out["lon_rot_360E"]=L360
            save = "/mnt/data/batch_rotate.csv"
        else:
            return f"Unknown operation '{op}'.", None
        out.to_csv(save, index=False)
        return f"Processed {len(out)} rows. Saved to {save.split('/')[-1]}", save
    except Exception as e:
        return f"Exception: {e}", None

# -----------------------
# Equal-area plots (simple)
# -----------------------
def plot_equal_area_dirs(I_deg, D_deg):
    colat = 90.0 - I_deg
    r = math.sqrt(2.0) * math.sin(d2r(colat)/2.0)
    x = r * math.sin(d2r(D_deg)); y = r * math.cos(d2r(D_deg))
    fig, ax = plt.subplots()
    circ = plt.Circle((0,0), 1.0, fill=False); ax.add_artist(circ)
    ax.plot(x, y, marker='o', linestyle='None')
    ax.set_aspect('equal', 'box'); ax.set_xlim([-1.05, 1.05]); ax.set_ylim([-1.05, 1.05])
    ax.set_xticks([]); ax.set_yticks([]); ax.set_title("Equal-area (direction)")
    buf = io.BytesIO(); fig.savefig(buf, format='png', bbox_inches='tight', dpi=150); plt.close(fig); buf.seek(0)
    return buf

def plot_equal_area_pole(lat_deg, lon_deg):
    colat = 90.0 - lat_deg
    r = math.sqrt(2.0) * math.sin(d2r(colat)/2.0)
    x = r * math.sin(d2r(lon_deg)); y = r * math.cos(d2r(lon_deg))
    fig, ax = plt.subplots()
    circ = plt.Circle((0,0), 1.0, fill=False); ax.add_artist(circ)
    ax.plot(x, y, marker='^', linestyle='None')
    ax.set_aspect('equal', 'box'); ax.set_xlim([-1.05, 1.05]); ax.set_ylim([-1.05, 1.05])
    ax.set_xticks([]); ax.set_yticks([]); ax.set_title("Equal-area (pole)")
    buf = io.BytesIO(); fig.savefig(buf, format='png', bbox_inches='tight', dpi=150); plt.close(fig); buf.seek(0)
    return buf

# -----------------------
# Test suite
# -----------------------
def run_tests():
    logs = []
    def check(name, got, exp, tol=1e-2):
        ok = abs(got - exp) <= tol
        logs.append(f"{name}: {'OK' if ok else 'FAIL'} (got {got:.4f}, exp {exp:.4f})")
        return ok
    # SOMMAMTX case validato
    lat, lon, _, ang = somma_mtx(22.0,124.0,-89.0, 56.0,323.0,87.0)
    check("SOMMAMTX lat", lat, 8.61); check("SOMMAMTX lon", lon, -17.47); check("SOMMAMTX ang", ang, 128.98)
    # ROTATE verificato
    rl, rlon, _ = rotate_pole(45.0,245.0, 40.0,56.0, 29.0)
    check("ROTATE lat", rl, 34.81); check("ROTATE lon", rlon, -79.35)
    # POLE2DIR sanity: pole(90,0) @ site(0,0) -> I~90, D~0
    I, D, p = pole2dir(90.0,0.0, 0.0,0.0); check("POLE2DIR I", I, 90.0); check("POLE2DIR D", D, 0.0)
    # PALAT da p
    check("PALAT from p", palat_from_pole(30.0), 60.0)
    return "\n".join(logs)

# -----------------------
# Build Gradio UI
# -----------------------
def build_interface():
    with gr.Blocks(title="Paleomag Toolkit — Full") as demo:
        gr.Markdown("# Paleomag Toolkit — Full")
        gr.Markdown("Angles in degrees. Longitudes → **[-180°, +180°]** (alt 0–360°E).  Positive rotation = counterclockwise (right-hand).")

        with gr.Tab("DIR2POLE"):
            with gr.Row():
                lat_site = gr.Number(label="Site Latitude (deg)", value=45.0)
                lon_site = gr.Number(label="Site Longitude (deg, East+)", value=9.0)
                inc_obs = gr.Number(label="Inclination I (deg)", value=60.0)
                dec_obs = gr.Number(label="Declination D (deg)", value=30.0)
                a95 = gr.Number(label="α95 (deg, optional)", value=5.0)
            run = gr.Button("Run"); reset = gr.Button("Reset")
            with gr.Row():
                out_lat = gr.Textbox(label="Pole Latitude", interactive=False)
                out_lon = gr.Textbox(label="Pole Longitude  (alt 0–360E)", interactive=False)
                out_p = gr.Textbox(label="Angular distance p", interactive=False)
                out_extra = gr.Textbox(label="Uncertainty summary", interactive=False)
            run.click(ui_DIR2POLE, inputs=[lat_site, lon_site, inc_obs, dec_obs, a95], outputs=[out_lat, out_lon, out_p, out_extra])
            reset.click(lambda : (45.0, 9.0, 60.0, 30.0, 5.0), outputs=[lat_site, lon_site, inc_obs, dec_obs, a95])
            with gr.Row():
                plot_btn = gr.Button("Plot pole (equal-area)"); plot_img = gr.Image(label="Plot", interactive=False)
            def _plot_dir2pole(lat_site, lon_site, inc_obs, dec_obs, a95):
                lat_p, lon_p, _ = dir2pole(lat_site, lon_site, inc_obs, dec_obs)
                return plot_equal_area_pole(lat_p, lon_p)
            plot_btn.click(_plot_dir2pole, inputs=[lat_site, lon_site, inc_obs, dec_obs, a95], outputs=[plot_img])

        with gr.Tab("POLE2DIR"):
            with gr.Row():
                lat_p = gr.Number(label="Pole Latitude (deg)", value=70.0)
                lon_p = gr.Number(label="Pole Longitude (deg, East+)", value=120.0)
                lat_s = gr.Number(label="Site Latitude (deg)", value=40.0)
                lon_s = gr.Number(label="Site Longitude (deg, East+)", value=12.0)
                A95 = gr.Number(label="Pole A95 (deg, optional)", value=5.0)
            run2 = gr.Button("Run"); reset2 = gr.Button("Reset")
            with gr.Row():
                out_I = gr.Textbox(label="Expected Inclination I", interactive=False)
                out_D = gr.Textbox(label="Expected Declination D", interactive=False)
                out_p2 = gr.Textbox(label="Angular distance p", interactive=False)
                out_extra2 = gr.Textbox(label="Uncertainty summary", interactive=False)
            run2.click(ui_POLE2DIR, inputs=[lat_p, lon_p, lat_s, lon_s, A95], outputs=[out_I, out_D, out_p2, out_extra2])
            reset2.click(lambda : (70.0,120.0,40.0,12.0,5.0), outputs=[lat_p, lon_p, lat_s, lon_s, A95])
            with gr.Row():
                plot_btn2 = gr.Button("Plot direction (equal-area)"); plot_img2 = gr.Image(label="Plot", interactive=False)
            def _plot_pole2dir(lat_p, lon_p, lat_s, lon_s, A95):
                I, D, _ = pole2dir(lat_p, lon_p, lat_s, lon_s)
                return plot_equal_area_dirs(I, D)
            plot_btn2.click(_plot_pole2dir, inputs=[lat_p, lon_p, lat_s, lon_s, A95], outputs=[plot_img2])

        with gr.Tab("UNCERTAINTY"):
            with gr.Row():
                a95_in = gr.Number(label="α95 (deg)", value=10.0)
                Imean = gr.Number(label="Mean Inclination I", value=45.0)
            run3 = gr.Button("Run"); reset3 = gr.Button("Reset")
            with gr.Row():
                out_dI = gr.Textbox(label="ΔI (quick-look)", interactive=False)
                out_dD = gr.Textbox(label="ΔD (quick-look)", interactive=False)
            run3.click(ui_UNCERT_dir, inputs=[a95_in, Imean], outputs=[out_dI, out_dD])
            reset3.click(lambda : (10.0,45.0), outputs=[a95_in, Imean])

        with gr.Tab("RFTEST"):
            with gr.Row():
                I_obs = gr.Number(label="Observed I", value=40.0)
                D_obs = gr.Number(label="Observed D", value=10.0)
                I_exp = gr.Number(label="Expected I", value=50.0)
                D_exp = gr.Number(label="Expected D", value=20.0)
                a95_rf = gr.Number(label="α95 (optional)", value=10.0)
            run4 = gr.Button("Run"); reset4 = gr.Button("Reset")
            with gr.Row():
                out_R = gr.Textbox(label="R (vertical rotation)", interactive=False)
                out_F = gr.Textbox(label="F (flattening)", interactive=False)
                out_rf = gr.Textbox(label="Uncertainty summary", interactive=False)
            run4.click(ui_RFTEST, inputs=[I_obs, D_obs, I_exp, D_exp, a95_rf], outputs=[out_R, out_F, out_rf])
            reset4.click(lambda : (40.0,10.0,50.0,20.0,10.0), outputs=[I_obs, D_obs, I_exp, D_exp, a95_rf])

        with gr.Tab("POLESHIFT"):
            with gr.Row():
                lat_obs = gr.Number(label="Observed Pole Lat", value=65.0)
                lon_obs = gr.Number(label="Observed Pole Lon", value=100.0)
                lat_ref = gr.Number(label="Reference Pole Lat", value=60.0)
                lon_ref = gr.Number(label="Reference Pole Lon", value=110.0)
            with gr.Row():
                lat_site_ps = gr.Number(label="Site Lat (optional)", value=40.0)
                lon_site_ps = gr.Number(label="Site Lon (optional)", value=10.0)
            run5 = gr.Button("Run"); reset5 = gr.Button("Reset")
            with gr.Row():
                out_dp = gr.Textbox(label="Δp or pole-to-pole arc", interactive=False)
                out_pobs = gr.Textbox(label="p_obs", interactive=False)
                out_pref = gr.Textbox(label="p_ref", interactive=False)
            run5.click(ui_POLESHIFT, inputs=[lat_obs, lon_obs, lat_ref, lon_ref, lat_site_ps, lon_site_ps],
                       outputs=[out_dp, out_pobs, out_pref])
            reset5.click(lambda : (65.0,100.0,60.0,110.0,40.0,10.0), outputs=[lat_obs, lon_obs, lat_ref, lon_ref, lat_site_ps, lon_site_ps])

        with gr.Tab("PALAT"):
            with gr.Row():
                I_in = gr.Number(label="Inclination I → Paleolat", value=45.0)
                p_in = gr.Number(label="Polar distance p → Paleolat", value=30.0)
            run6a = gr.Button("Compute from I"); run6b = gr.Button("Compute from p")
            reset6a = gr.Button("Reset I"); reset6b = gr.Button("Reset p")
            out_pal_I = gr.Textbox(label="Paleolat from I", interactive=False)
            out_pal_p = gr.Textbox(label="Paleolat from p", interactive=False)
            run6a.click(ui_PALAT_from_dir, inputs=[I_in], outputs=[out_pal_I])
            run6b.click(ui_PALAT_from_pole, inputs=[p_in], outputs=[out_pal_p])
            reset6a.click(lambda : (45.0,), outputs=[I_in])
            reset6b.click(lambda : (30.0,), outputs=[p_in])

        with gr.Tab("SOMMAMTX"):
            with gr.Row():
                lat1 = gr.Number(label="Older: Lat", value=22.0)
                lon1 = gr.Number(label="Older: Lon", value=124.0)
                ang1 = gr.Number(label="Older: Angle", value=-89.0)
                lat2 = gr.Number(label="Younger: Lat", value=56.0)
                lon2 = gr.Number(label="Younger: Lon", value=323.0)
                ang2 = gr.Number(label="Younger: Angle", value=87.0)
            run7 = gr.Button("Run"); reset7 = gr.Button("Reset")
            with gr.Row():
                out_lat_c = gr.Textbox(label="Cumulative Lat", interactive=False)
                out_lon_c = gr.Textbox(label="Cumulative Lon (alt 0–360E)", interactive=False)
                out_ang_c = gr.Textbox(label="Cumulative Angle (>0)", interactive=False)
            with gr.Row():
                out_lat_alt = gr.Textbox(label="Antipode Lat", interactive=False)
                out_lon_alt = gr.Textbox(label="Antipode Lon", interactive=False)
                out_ang_alt = gr.Textbox(label="Antipode Angle (<0)", interactive=False)
            run7.click(ui_SOMMAMTX, inputs=[lat1, lon1, ang1, lat2, lon2, ang2],
                       outputs=[out_lat_c, out_lon_c, out_ang_c, out_lat_alt, out_lon_alt, out_ang_alt])
            reset7.click(lambda : (22.0,124.0,-89.0,56.0,323.0,87.0), outputs=[lat1, lon1, ang1, lat2, lon2, ang2])

        with gr.Tab("ROTATE"):
            gr.Markdown("Rotate paleomagnetic pole(s) about an Euler pole (lat, lon, ω). Positive ω = counterclockwise (right-hand).")
            with gr.Row():
                lat_p = gr.Number(label="Paleopole Lat", value=30.0)
                lon_p = gr.Number(label="Paleopole Lon", value=100.0)
                e_lat = gr.Number(label="Euler Lat", value=10.0)
                e_lon = gr.Number(label="Euler Lon", value=250.0)
                omega = gr.Number(label="Angle ω", value=25.0)
            run8 = gr.Button("Rotate single"); reset8 = gr.Button("Reset")
            with gr.Row():
                out_rlat = gr.Textbox(label="Rotated Lat", interactive=False)
                out_rlon = gr.Textbox(label="Rotated Lon (alt 0–360E)", interactive=False)
            run8.click(ui_ROTATE_single, inputs=[lat_p, lon_p, e_lat, e_lon, omega], outputs=[out_rlat, out_rlon])
            reset8.click(lambda : (30.0,100.0,10.0,250.0,25.0,None), outputs=[lat_p, lon_p, e_lat, e_lon, omega, gr.File()])
            gr.Markdown("Batch rotation from CSV (columns 'lat'/'latitude' and 'lon'/'longitude').")
            in_file = gr.File(label="Upload CSV of poles", file_types=[".csv"])
            run8b = gr.Button("Rotate batch and export CSV")
            out_msg = gr.Textbox(label="Status", interactive=False)
            out_file = gr.File(label="Download rotated_poles.csv")
            run8b.click(ui_ROTATE_batch, inputs=[in_file, e_lat, e_lon, omega], outputs=[out_msg, out_file])

        # ---- BATCH (corretto: niente None negli inputs) ----
        with gr.Tab("BATCH"):
            gr.Markdown("Batch CSV operations. Choose operation and provide required columns in the CSV.")
            op = gr.Dropdown(choices=["DIR2POLE","POLE2DIR","SOMMAMTX","ROTATE"], value="DIR2POLE", label="Operation")
            in_file_b = gr.File(label="Upload CSV", file_types=[".csv"])
            gr.Markdown("For SOMMAMTX: lat1,lon1,ang1,lat2,lon2,ang2.  DIR2POLE: site_lat,site_lon,inc,dec.  POLE2DIR: pole_lat,pole_lon,site_lat,site_lon.  ROTATE: lat,lon (set Euler below).")
            with gr.Row():
                A_lat = gr.Number(label="Euler/Op A Lat (for ROTATE)", value=10.0)
                A_lon = gr.Number(label="Euler/Op A Lon (for ROTATE)", value=250.0)
                A_ang = gr.Number(label="Euler/Op A Angle (for ROTATE)", value=25.0)
                # Placeholders nascosti per mantenere la firma (mai usati realmente qui)
                S_lat = gr.Number(value=0.0, visible=False)
                S_lon = gr.Number(value=0.0, visible=False)
                B_lat = gr.Number(value=0.0, visible=False)
                B_lon = gr.Number(value=0.0, visible=False)
                B_ang = gr.Number(value=0.0, visible=False)
            runB = gr.Button("Process CSV")
            out_msgB = gr.Textbox(label="Status", interactive=False)
            out_fileB = gr.File(label="Download result CSV")
            runB.click(ui_BATCH,
                       inputs=[in_file_b, op, S_lat, S_lon, A_lat, A_lon, A_ang, B_lat, B_lon, B_ang],
                       outputs=[out_msgB, out_fileB])

        with gr.Tab("TEST SUITE"):
            runT = gr.Button("Run tests")
            outT = gr.Textbox(label="Report", lines=12, interactive=False)
            runT.click(lambda : run_tests(), inputs=[], outputs=[outT])

    return demo

if __name__ == "__main__":
    demo = build_interface()
    demo.launch(share=True)


Overwriting paleomag_colab_full.py


In [ ]:
!python paleomag_colab_full.py

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://d46e0a330cf713d94a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
